In [2]:
# import, etc
import pandas as pd
import math
inputFile = "../Raingauge.v1/2020-09-26-Rain gauge-raw form.csv"

# Catching up with CoCoRaHS
In 2020, for a number of reasons, i got out of the habit of recording to CoCoRaHS as well as my Airtable record. This notebook is to help me catch up in time to be included in the October summary calculations.

## Message re summary reports

The end of the water year means we will be running new summary reports for every participating CoCoRaHS station.  Previous summaries for your station are found on your "My Account" page linked along the top-menu of our website (login required).  You can also look at other stations' water year summaries found in our 'View Data' page.  These summary data are the crown jewels of CoCoRaHS, so please make sure the data for your station is as complete and accurate as possible.  We will be running the first round of Water Year Summary reports around October 7.  Then we will wait a few weeks for all of us to catch up on entering any late or missing reports and/or to correct any errors we find in our data entries.  (A reminder - you can always review and edit your data from the links in the left-hand menu of your data entry page.)  We’ll then update the reports again around October 31.  These reports are "static" - meaning they do not update automatically when you add or edit data.  This is why we generate it once in early October and then again at the end of the month.  Even if you’ve only been part of CoCoRaHS for a part of the year, we will still do a thorough summary of your data.  Thanks so much for helping make this possible.

## Attempt to use iframe in notebook seems to fail

<iframe class="airtable-embed" src="https://airtable.com/embed/shr0qeDlQyZvvLDNt?backgroundColor=green&viewControls=on" frameborder="0" onmousewheel="" width="100%" height="533" style="background: transparent; border: 1px solid #ccc;"></iframe>

## How can i spread out the work?

### Dataset 1
- [ ] 2/29/2020 08:26        Dawn  clear & frosty. Observation overcast. 
- [ ] 3/1/2020 07:00    0.00   T  At 8:47 on the 29th graupel! Just a minute. then clearing.
- [ ] 3/2/2020 07:00    0.00    
- [ ] 3/3/2020 07:00    0.00    
- [ ] 3/4/2020 08:30    0.22    
- [ ] 3/6/2020 07:45    0.05    , 05 no frost 745am mostly cloudy
- [ ] 3/7/2020 08:50        clear, windy
- [ ] 8 & 9 - zeros
- [ ] 8 & 9 - zeros

### Remaining data

In [13]:
# How much needs to be reported?
startDate = "2020-03-11" # have an entry in for 2020-03-10
endDate = "2020-09-26"
dataSet1 = 9
dataSet1Start = "2020-02-29"
dataSet1Stop = "2020-03-09"
daysToReport = (pd.Timestamp(endDate) - pd.Timestamp(startDate)).days + dataSet1

In [15]:
# How long do i have to get it done?
beginDate = "2020-09-26"
deadline = "2020-10-06"
daysToWork = (pd.Timestamp(deadline) - pd.Timestamp(beginDate)).days

In [17]:
# How to spread it evenly (and rounding up to integer days)
entriesPerDay = math.ceil(daysToReport/daysToWork)

In [19]:
# Read in the data & prepare
with open(inputFile) as f:
    sourceDF = pd.read_csv(f)
# convert Observation to datetime
sourceDF['Observation'] = pd.to_datetime(sourceDF['Observation'])
# Limit the columns in the dataframe to those required
sourceDF = sourceDF[['Observation',"precip (in)","Notes","spilled" ,"Trace"]]
sourceDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Observation  147 non-null    datetime64[ns]
 1   precip (in)  141 non-null    float64       
 2   Notes        119 non-null    object        
 3   spilled      3 non-null      object        
 4   Trace        6 non-null      object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 5.9+ KB


There are {{len(sourceDF[sourceDF['Trace'].notna()])}} records of Trace amounts. (NaN vs "checked")

There are {{len(sourceDF[sourceDF['spilled'].notna()])}} records of spilled amounts. (NaN vs "checked")

So those fields are necessary in for reporting

In [22]:
# Develop the task dictionary
taskDict = {}

To get done in time, {{entriesPerDay}} entries per set for {{daysToWork}} sets.  

Data Set 1 has {{dataSet1}} entries. So, the first set is Data Set 1 plus {{startDate}}  and {{entriesPerDay - dataSet1}} more report days. 

The following have {{entriesPerDay}} days per set.

In [25]:
# Day 1 
i = 1
taskDict[i] = {}
taskDict[i]['startDate'] = pd.Timestamp(dataSet1Start)
taskDict[i]['endDate'] = pd.Timestamp(startDate) + pd.Timedelta((entriesPerDay - dataSet1),'days')

# Two dataframes, concatenated:
sTS = pd.Timestamp(dataSet1Start)
eTS = pd.Timestamp(dataSet1Stop)
mask = (sourceDF['Observation'] >= sTS ) & (sourceDF['Observation'] <= eTS)
aDF = sourceDF.loc[mask]

sTS = pd.Timestamp(startDate)
eTS = taskDict[i]['endDate']
mask = (sourceDF['Observation'] >= sTS ) & (sourceDF['Observation'] <= eTS)
bDF = sourceDF.loc[mask]


taskDict[i]['DF'] = pd.concat([aDF,bDF])


{{taskDict[i]['startDate']}} to {{taskDict[i]['endDate']}}

In [28]:
# Days 2 through end
i = 2

while i <= daysToWork:
    taskDict[i] = {}
    taskDict[i]['startDate'] = taskDict[i-1]['endDate'] + pd.Timedelta(1,'days')
    taskDict[i]['endDate'] = taskDict[i]['startDate'] + pd.Timedelta((entriesPerDay),'days')
    
    sTS = taskDict[i]['startDate']
    eTS = taskDict[i]['endDate'] + pd.Timedelta(5,'days')
    mask = (sourceDF['Observation'] >= sTS ) & (sourceDF['Observation'] <= eTS)
    sourceDF.loc[mask]
    i += 1

In [31]:
i = 1
print(taskDict[i]['startDate'])
print(taskDict[i]['endDate'])
# ValueError: index must be monotonic increasing or decreasing
# sourceDF.loc[taskDict[i]['startDate']:taskDict[i]['endDate']]

#greater than the start date and smaller than the end date
sTS = taskDict[i]['startDate']
eTS = taskDict[i]['endDate'] + pd.Timedelta(5,'days')
mask = (sourceDF['Observation'] >= sTS ) & (sourceDF['Observation'] <= eTS)
sourceDF.loc[mask]

2020-02-29 00:00:00
2020-03-23 00:00:00


,Observation,precip (in),Notes,spilled,Trace
46,2020-02-29 08:26:00,NaN,Dawn clear & frosty. Observation overcast. At...,NaN,checked
47,2020-03-01 07:00:00,0.00,NaN,NaN,NaN
48,2020-03-02 07:00:00,0.00,NaN,NaN,NaN
49,2020-03-03 07:00:00,0.00,NaN,NaN,NaN
50,2020-03-04 08:30:00,0.22,NaN,NaN,NaN
51,2020-03-06 07:45:00,0.05,", 05 no frost 745am mostly cloudy",NaN,NaN
52,2020-03-07 08:50:00,NaN,"clear, windy",NaN,checked
53,2020-03-10 08:15:00,0.00,Overcast & glum (cocorhas done),NaN,NaN
54,2020-03-11 08:00:00,0.02,soft morning. no frost,NaN,NaN
55,2020-03-16 08:30:00,0.05,MULTIDAY -- and mainly the first day in the mu...,NaN,NaN


In [33]:
a = pd.Period(startDate, 'D')
b = pd.Period(endDate,'D')
daily = pd.period_range(start=a, end=b, freq='D')

In [35]:
tempDF = sourceDF
tempDF= tempDF.set_index('Observation',drop=False)
tempDF.groupby(tempDF.index.to_period('D')).count()

,Observation,precip (in),Notes,spilled,Trace
Observation,,,,,
2020-01-01,1,1,1,0,0
2020-01-02,1,1,1,0,0
2020-01-03,1,1,0,0,0
2020-01-04,1,1,1,0,0
2020-01-05,1,1,1,0,0
...,...,...,...,...,...
2020-09-18,1,1,1,0,0
2020-09-19,1,1,1,0,0
2020-09-23,1,1,1,0,0


## Well, that's not so helpful

I can't figure out how to identify missing days or when multiday reports are needed,